In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('отзывы за лето.xls')

In [3]:
df.drop('Date', axis=1, inplace=True)

In [4]:
df.head(5)

,Rating,Content
0,5,It just works!
1,4,В целом удобноное приложение...из минусов хотя...
2,5,Отлично все
3,5,Стал зависать на 1% работы антивируса. Дальше ...
4,5,"Очень удобно, работает быстро."


In [5]:
df.rename(columns={"Rating": "class", "Content": "text"}, inplace=True)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
max_words = 2000 
max_len = 20
num_classes = 6

# Training
epochs = 20
batch_size = 16
print_batch_n = 100

In [8]:
df_train, for_test = train_test_split(df, test_size=0.35,  shuffle=True)
df_test, df_val = train_test_split(for_test, test_size=0.42, shuffle=True)

In [9]:
df_train.head(5)

,class,text
17526,1,Только Шаблоны
17770,4,нехочет работать с NFC
11647,5,"Спасибо, что вы всегда рядом!"
3525,5,Удобное и понятное приложение. 4-ка только за ...
15119,4,Удобное приложение


In [10]:
df_test.head(5)

,class,text
2872,5,Только установил но уже нравиться по-моему оче...
5039,5,Программа супер. Очень выручает.
749,5,Все быстро и удобно
15177,5,Ок
18527,5,"Перевожу, оплату делаю, все хорошо!"


In [11]:
df_val.head()

,class,text
15369,5,"Все отлично, но с шаблонами бывают проблемы, з..."
9842,4,"Предложением пользуюсь давно. Очень нравиться,..."
2335,3,"Изначально всё устраивало, но вот уже нескольк..."
11208,4,Удобный сервис
15875,5,Настоящий верный помошник)


In [12]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [13]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

# по сути тут не нужен препроцессинг, кроме токинизаии и лемматизации
# оставлю от твиттера.
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

<ipython-input-13-254a6bd4a4ca>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['text'] = df_train['text'].apply(preprocess_text)
<ipython-input-13-254a6bd4a4ca>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['text'] = df_val['text'].apply(preprocess_text)
<ipython-input-13-254a6bd4a4ca>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [14]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [15]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zald\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [17]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [18]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [19]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [20]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [21]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [22]:
x_train.shape, max_len

((13428, 20), 20)

In [23]:
x_train[2]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [24]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [25]:
import tensorflow as tf
y_train = tf.keras.utils.to_categorical(df_train["class"], num_classes)
y_val = tf.keras.utils.to_categorical(df_val["class"], num_classes)

In [26]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())

# model.add(Dense(1))
# model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [27]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [28]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
756/756 [==============================] - 21s 7ms/step - loss: 0.8457 - accuracy: 0.7356 - val_loss: 0.6647 - val_accuracy: 0.7796


In [29]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

190/190 [==============================] - 1s 3ms/step - loss: 0.6601 - accuracy: 0.7791


Test score: 0.6601474285125732
Test accuracy: 0.7791309952735901


In [30]:
def different_convalutions(conv=(2,2)):
    results = []
    for temp_conv in range(conv[0], conv[1]+1):
        model = Sequential()
        model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
        model.add(Conv1D(128, temp_conv))
        model.add(Activation("relu"))
        model.add(GlobalMaxPool1D())
        model.add(Activation("relu"))
        model.add(Dense(num_classes))
        model.add(Activation('softmax'))

        model.compile(
            loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy']
        )
        tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
        early_stopping=EarlyStopping(monitor='val_loss')  


        history = model.fit(x_train, y_train,
                            batch_size=batch_size,
                            epochs=epochs,
                            verbose=1,
                            validation_split=0.1,
                            callbacks=[tensorboard, early_stopping])
        # results.append((history.history, temp_conv))
        results.append(history.history)
    return results, [i for i in range(conv[0], conv[1]+1)]
        

In [31]:
res = different_convalutions((1,15))

Epoch 1/20
756/756 [==============================] - 7s 8ms/step - loss: 0.8907 - accuracy: 0.7263 - val_loss: 0.6543 - val_accuracy: 0.7796
Epoch 1/20
756/756 [==============================] - 7s 8ms/step - loss: 0.8792 - accuracy: 0.7220 - val_loss: 0.6541 - val_accuracy: 0.7684
Epoch 1/20
756/756 [==============================] - 7s 8ms/step - loss: 0.8460 - accuracy: 0.7394 - val_loss: 0.6434 - val_accuracy: 0.7766
Epoch 1/20
756/756 [==============================] - 7s 8ms/step - loss: 0.8604 - accuracy: 0.7312 - val_loss: 0.6588 - val_accuracy: 0.7707
Epoch 1/20
756/756 [==============================] - 7s 8ms/step - loss: 0.8442 - accuracy: 0.7319 - val_loss: 0.6640 - val_accuracy: 0.7811
Epoch 1/20
756/756 [==============================] - 7s 8ms/step - loss: 0.8156 - accuracy: 0.7398 - val_loss: 0.6489 - val_accuracy: 0.7744
Epoch 1/20
756/756 [==============================] - 7s 8ms/step - loss: 0.8415 - accuracy: 0.7306 - val_loss: 0.6642 - val_accuracy: 0.7729
Epoch 

In [32]:
pd.DataFrame(res[0], index=res[1])

,loss,accuracy,val_loss,val_accuracy
1,[0.7371109127998352],[0.7542408108711243],[0.654323399066925],[0.7795979380607605]
2,[0.7318294644355774],[0.7552337646484375],[0.6541197299957275],[0.7684288620948792]
3,[0.7236555218696594],[0.7580471634864807],[0.6433730125427246],[0.7766194939613342]
4,[0.7326981425285339],[0.7556474804878235],[0.6587592959403992],[0.7706627249717712]
5,[0.717983603477478],[0.757716178894043],[0.6640352606773376],[0.7810871005058289]
6,[0.7150466442108154],[0.7589573860168457],[0.6488779187202454],[0.7743856906890869]
7,[0.7198755741119385],[0.7561439871788025],[0.6642069220542908],[0.7728965282440186]
8,[0.7163037061691284],[0.7601158618927002],[0.6711126565933228],[0.7684288620948792]
9,[0.7198479175567627],[0.7577989101409912],[0.6623789072036743],[0.7699180841445923]
10,[0.7207155823707581],[0.7576334476470947],[0.6509768962860107],[0.7788532972335815]


Размер свертки практически не влияет на результат, все результаты в пределах погрешности. 

In [33]:
with open('180/model.txt', 'r', encoding='utf8') as f:
    pretrained_words = f.readlines()
pretrained_words.pop(0)

'189193 300\n'

In [34]:
pretrained_words[1].split()[0].split('_')[0]

'быть'

In [35]:
np.asarray(pretrained_words[1].split()[1:], dtype='float32').shape

(300,)

In [36]:
counter = 0
embeddings_index = {}
for pre_word in pretrained_words:
    counter += 1
    word = pre_word.split()[0].split('_')[0]
    coefs = np.asarray(pre_word.split()[1:], dtype='float32')
    embeddings_index[word] = coefs
print(counter)

189193


In [37]:
EMBEDDING_DIM = 300

In [38]:
embedding_matrix = np.zeros((len(vocabulary) + 1, EMBEDDING_DIM))
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [39]:
embedding_layer = Embedding(input_dim=len(vocabulary) + 1,
                            output_dim=EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)

In [40]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(50))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 20, 300)           600000    
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 16, 128)           192128    
_________________________________________________________________
activation_47 (Activation)   (None, 16, 128)           0         
_________________________________________________________________
global_max_pooling1d_16 (Glo (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 50)                6450      
_________________________________________________________________
activation_48 (Activation)   (None, 50)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 6)               

In [41]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [42]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
756/756 [==============================] - 6s 6ms/step - loss: 0.9467 - accuracy: 0.7095 - val_loss: 0.7261 - val_accuracy: 0.7446


С предобученым эмбедингом результаты получаются хуже, вполне возможно это связано с тем что слишком много уникальных(неправильных, тех же смайликов) слов находится в наших отзывах. Они просто нули в нашей модели.